In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import numpy as np
import pandas as pd

In [33]:
data = pd.read_csv("/content/drive/MyDrive/Zipbob_timeseries/pricedb예제data.csv", encoding='cp949')

# 농수산 가격 예제 불러오기

In [34]:
data

,itemnm,kindnm,regdate,price
0,팥,붉은 팥(국산)(1kg),1996-01-03,"3,375"
1,팥,붉은 팥(국산)(1kg),1996-01-04,"3,375"
2,팥,붉은 팥(국산)(1kg),1996-01-05,"3,375"
3,팥,붉은 팥(국산)(1kg),1996-01-08,"3,375"
4,팥,붉은 팥(국산)(1kg),1996-01-09,"3,375"
...,...,...,...,...
335770,가지,가지(1kg),2011-04-21,"3,400"
335771,가지,가지(1kg),2011-04-22,"3,400"
335772,가지,가지(1kg),2011-04-25,"3,400"
335773,가지,가지(1kg),2011-04-26,"3,400"


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335775 entries, 0 to 335774
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   itemnm   335775 non-null  object
 1   kindnm   335775 non-null  object
 2   regdate  335775 non-null  object
 3   price    335775 non-null  object
dtypes: object(4)
memory usage: 10.2+ MB


In [36]:
data.shape # 중복행제거 21,504개

(335775, 4)

In [37]:
# 과거날짜, 최근날짜 확인
data['regdate'].min(), data['regdate'].max()

('1996-01-03', '2021-09-16')

In [38]:
# 등록된 야채 종류 itemnm기준
data['itemnm'].unique(), len(data['itemnm'].unique())

(array(['팥', '녹두', '고구마', '양배추', '시금치', '얼갈이배추', '수박', '참외', '토마토', '딸기',
        '당근', '열무', '건고추', '풋고추', '붉은고추', '양파', '파', '생강', '가지', '미나리',
        '깻잎', '부추', '피망', '파프리카', '멜론', '느타리버섯', '팽이버섯', '새송이버섯', '감귤',
        '단감', '레몬', '체리', '망고', '건멸치', '건오징어', '김', '건미역', '굴', '전복', '쌀',
        '찹쌀', '콩', '감자', '배추', '상추', '호박', '무', '피마늘', '깐마늘(국산)', '방울토마토',
        '참깨', '들깨', '땅콩', '사과', '배', '복숭아', '포도', '바나나', '참다래', '파인애플',
        '오렌지', '고등어', '갈치', '물오징어', '북어', '새우', '메밀', '오이'], dtype=object), 68)

In [39]:
# numpy의 array 말고 리스트로 만들기
itemlist = list(data['itemnm'].unique())

In [40]:
# 데이터 등록된 전체 날짜 df만들기
total_date = pd.date_range(start='1996-01-03', end='2021-09-16')
from pandas import DataFrame
total_date = DataFrame(total_date)

## 1. 중복 날짜 갖는 야채 있는지 파악

In [44]:
가지날짜 = data[data['itemnm']=='가지']['regdate'].values #numpy ndarray
가지날짜테스트 = []
for i in range(len(가지날짜)):
    가지날짜테스트.append(가지날짜[i])

In [45]:
print(가지날짜테스트[-10:])
print(가지날짜테스트[:10])

['2011-04-14', '2011-04-15', '2011-04-18', '2011-04-19', '2011-04-20', '2011-04-21', '2011-04-22', '2011-04-25', '2011-04-26', '2011-04-27']
['1996-04-22', '1996-04-23', '1996-04-24', '1996-04-25', '1996-04-26', '1996-04-29', '1996-04-30', '1996-05-02', '1996-05-03', '1996-05-06']


In [ ]:
# count.clear()
count={}
for i in 가지날짜테스트:
    try: count[i] += 1
    except: count[i]=1
if 2 in list(count.values()) :
    print(True) # 중복날짜 있습니다
else :
    print(False) # 중복날짜 없습니다

True


가지랑 시금치가 다르다

In [46]:
시금치날짜 = data[data['itemnm']=='시금치']['regdate'].values #numpy ndarray
시금치날짜테스트 = []
for i in range(len(시금치날짜)):
    시금치날짜테스트.append(시금치날짜[i])

In [47]:
count.clear()
count={}
for i in 시금치날짜테스트:
    try: count[i] += 1
    except: count[i]=1
if 2 in list(count.values()) :
    print(True) # 중복날짜 있습니다
else :
    print(False) # 중복날짜 없습니다

False


### 가지 날짜처럼 날짜 중복값이 하나라도 2개 이상인 거 찾기

In [48]:
# 버전1 : 실행속도 느림 
def has_duplicates(seq):
    return len(seq) != len(set(seq))
# has_duplicates(야채날짜리스트)
중복날짜야채리스트 = []
for x in itemlist :
    야채날짜 = data[data['itemnm'] == x]['regdate'].values #numpy ndarray
    야채날짜리스트 = []
    for y in range(len(야채날짜)):
        야채날짜리스트.append(야채날짜[y])
    if has_duplicates(야채날짜리스트) == True :
        중복날짜야채리스트.append(x)
print(중복날짜야채리스트)

['팥', '얼갈이배추', '참외', '토마토', '딸기', '열무', '건고추', '양파', '가지', '피망', '파프리카', '멜론', '느타리버섯', '팽이버섯', '새송이버섯', '감귤', '단감', '레몬', '체리', '망고', '쌀', '찹쌀', '콩', '감자', '배추', '상추', '호박', '무', '피마늘', '깐마늘(국산)']


### 직접 날짜 중복되는 농수산 종류 출력하기

In [49]:
more2itemlist = []
for i in itemlist :
    야채별날짜 = list(data[data['itemnm']==i]['regdate'].values)
    count={}
    for m in 야채별날짜:
        try: count[m] += 1
        except: count[m] = 1
    if 2 in list(count.values()) :
        more2itemlist.append(i)
    count.clear()

In [50]:
print(more2itemlist) # 날짜중복값 있는 야채 리스트 출력

['팥', '얼갈이배추', '참외', '토마토', '딸기', '열무', '건고추', '양파', '가지', '피망', '파프리카', '멜론', '느타리버섯', '팽이버섯', '새송이버섯', '감귤', '단감', '레몬', '체리', '망고', '쌀', '찹쌀', '콩', '감자', '상추', '호박', '피마늘']


### 파악
- 야채-날짜별 중복값도 있고, itemnm 종류 중 kindnm이 겹치는 것
- 모든컬럼 동일값인 중복행 제거는 drop_duplicates로 실행하기
- 나머지 야채 중에서 감자-감자종류들 많으면 kindnm이 다르니까
itemnm과 kindnm 조합으로 탐색하기

In [51]:
data[data['itemnm']=='무']['kindnm'].unique()

array(['봄(1kg)', '고랭지(1kg)', '월동(1kg)', '가을(1kg)'], dtype=object)

## item - kind 조합

In [52]:
# 등록된 야채 종류 itemnm기준
data['itemnm'].unique(), len(data['itemnm'].unique())

(array(['팥', '녹두', '고구마', '양배추', '시금치', '얼갈이배추', '수박', '참외', '토마토', '딸기',
        '당근', '열무', '건고추', '풋고추', '붉은고추', '양파', '파', '생강', '가지', '미나리',
        '깻잎', '부추', '피망', '파프리카', '멜론', '느타리버섯', '팽이버섯', '새송이버섯', '감귤',
        '단감', '레몬', '체리', '망고', '건멸치', '건오징어', '김', '건미역', '굴', '전복', '쌀',
        '찹쌀', '콩', '감자', '배추', '상추', '호박', '무', '피마늘', '깐마늘(국산)', '방울토마토',
        '참깨', '들깨', '땅콩', '사과', '배', '복숭아', '포도', '바나나', '참다래', '파인애플',
        '오렌지', '고등어', '갈치', '물오징어', '북어', '새우', '메밀', '오이'], dtype=object), 68)

In [53]:
# 등록된 야채 종류 kindnm기준
data['kindnm'].unique(), len(data['kindnm'].unique())

(array(['붉은 팥(국산)(1kg)', '국산(1kg)', '밤(1kg)', '양배추(1kg)', '시금치(1kg)',
        '얼갈이배추(1kg)', '수박(1개)', '참외(1kg)', '토마토(1kg)', '딸기(1kg)',
        '무세척(1kg)', '열무(1kg)', '화건(1kg)', '햇산화건(1kg)', '풋고추(1kg)',
        '붉은고추(1kg)', '양파(1kg)', '햇양파(1kg)', '대파(1kg)', '가지(1kg)',
        '미나리(1kg)', '깻잎(1kg)', '부추(1kg)', '청(1kg)', '파프리카(1kg)', '멜론(1kg)',
        '느타리버섯(1kg)', '팽이버섯(1kg)', '새송이버섯(1kg)', '감귤(1kg)', '노지(1kg)',
        '단감(1kg)', '수입(1kg)', '건멸치(1kg)', '건오징어(20마리)', '마른김(1속)',
        '건미역(1kg)', '굴(1kg)', '전복(1kg)', '일반계(1kg)', '햇일반계(1kg)',
        '흰 콩(국산)(1kg)', '수미(1kg)', '대지마(1kg)', '봄(1kg)', '고랭지(1kg)',
        '월동(1kg)', '적(1kg)', '애호박(20개)', '한지(1kg)', '깐마늘(대서)(1kg)',
        '햇깐마늘(대서)(1kg)', '깐마늘(남도)(1kg)', '햇깐마늘(남도)(1kg)', '방울토마토(1kg)',
        '백색(국산)(1kg)', '홍로(1kg)', '신고(1kg)', '백도(1kg)', '캠벨얼리(1kg)',
        '네이블 미국(1kg)', '생선(1kg)', '황태(10마리)', '흰다리(수입)(1kg)',
        '메밀(수입)(1kg)', '가을(1kg)', '가시계통(1kg)'], dtype=object), 67)

In [54]:
for i in itemlist :
    dataset = data[data['itemnm']==i]
    if len(dataset['kindnm'].unique()) >= 2 :
        print(i, dataset['kindnm'].unique())

건고추 ['화건(1kg)' '햇산화건(1kg)']
양파 ['양파(1kg)' '햇양파(1kg)']
감귤 ['감귤(1kg)' '노지(1kg)']
쌀 ['일반계(1kg)' '햇일반계(1kg)']
감자 ['수미(1kg)' '대지마(1kg)']
배추 ['봄(1kg)' '고랭지(1kg)' '월동(1kg)' '가을(1kg)']
상추 ['적(1kg)' '청(1kg)']
무 ['봄(1kg)' '고랭지(1kg)' '월동(1kg)' '가을(1kg)']
깐마늘(국산) ['깐마늘(대서)(1kg)' '햇깐마늘(대서)(1kg)' '깐마늘(남도)(1kg)' '햇깐마늘(남도)(1kg)']


#### 봄(1kg)처럼 배추/무 달라도 종류가 같아서 중복 카운트될 수 있음
kindnm을 unique로 뽑아내도, itemnm에 중복으로 들어감. 그래서 67,68처럼 숫자가 비슷하게 나오게 됨

In [55]:
data[data['kindnm']=='봄(1kg)']

,itemnm,kindnm,regdate,price
206239,배추,봄(1kg),2021-01-04,-
206240,배추,봄(1kg),2021-01-05,-
206241,배추,봄(1kg),2021-01-06,-
206242,배추,봄(1kg),2021-01-07,-
206243,배추,봄(1kg),2021-01-08,-
...,...,...,...,...
327005,무,봄(1kg),2021-09-10,-
327006,무,봄(1kg),2021-09-13,-
327007,무,봄(1kg),2021-09-14,-
327008,무,봄(1kg),2021-09-15,-


### itemnm-kindnm으로 셋 구분한 다음에 '-' 파악하기

## 2. 결측치 '-' 값 파악

In [56]:
priceisnull = data[data['price']=='-']

In [57]:
priceisnull.shape

(54858, 4)

가격 없는 row 54,858개 --> 확인

In [58]:
priceisnull['itemnm'].unique()

array(['건고추', '양파', '쌀', '감자', '배추', '무', '깐마늘(국산)'], dtype=object)

In [59]:
nullcnt = 0
null_itemkind = []
for i in range(len(itemlist)):
    dataset = data[data['itemnm'] == itemlist[i]] # 상위카테고리별 데이터셋
    kind_unique = list(dataset['kindnm'].unique())
    for k in range(len(kind_unique)) :
        dataset_kind = dataset[dataset['kindnm']==kind_unique[k]]
        kind_null = dataset_kind[dataset_kind['price']=='-']
        print(kind_unique[k], len(kind_null))
        if len(kind_null) > 0 :
            nullcnt += 1
            null_itemkind.append([itemlist[i], kind_unique[k], len(kind_null)])
    kind_unique=[]

붉은 팥(국산)(1kg) 0
국산(1kg) 0
밤(1kg) 0
양배추(1kg) 0
시금치(1kg) 0
얼갈이배추(1kg) 0
수박(1개) 0
참외(1kg) 0
토마토(1kg) 0
딸기(1kg) 0
무세척(1kg) 0
열무(1kg) 0
화건(1kg) 8
햇산화건(1kg) 6070
풋고추(1kg) 0
붉은고추(1kg) 0
양파(1kg) 76
햇양파(1kg) 6422
대파(1kg) 0
국산(1kg) 0
가지(1kg) 0
미나리(1kg) 0
깻잎(1kg) 0
부추(1kg) 0
청(1kg) 0
파프리카(1kg) 0
멜론(1kg) 0
느타리버섯(1kg) 0
팽이버섯(1kg) 0
새송이버섯(1kg) 0
감귤(1kg) 0
노지(1kg) 0
단감(1kg) 0
수입(1kg) 0
수입(1kg) 0
수입(1kg) 0
건멸치(1kg) 0
건오징어(20마리) 0
마른김(1속) 0
건미역(1kg) 0
굴(1kg) 0
전복(1kg) 0
일반계(1kg) 29
햇일반계(1kg) 6902
일반계(1kg) 0
흰 콩(국산)(1kg) 0
수미(1kg) 0
대지마(1kg) 3086
봄(1kg) 5027
고랭지(1kg) 4748
월동(1kg) 2856
가을(1kg) 4327
적(1kg) 0
청(1kg) 0
애호박(20개) 0
봄(1kg) 5107
고랭지(1kg) 4869
월동(1kg) 1299
가을(1kg) 3690
한지(1kg) 0
깐마늘(대서)(1kg) 0
햇깐마늘(대서)(1kg) 169
깐마늘(남도)(1kg) 4
햇깐마늘(남도)(1kg) 169
방울토마토(1kg) 0
백색(국산)(1kg) 0
국산(1kg) 0
국산(1kg) 0
홍로(1kg) 0
신고(1kg) 0
백도(1kg) 0
캠벨얼리(1kg) 0
수입(1kg) 0
국산(1kg) 0
수입(1kg) 0
네이블 미국(1kg) 0
생선(1kg) 0
생선(1kg) 0
생선(1kg) 0
황태(10마리) 0
흰다리(수입)(1kg) 0
메밀(수입)(1kg) 0
가시계통(1kg) 0


In [60]:
print('결측야채개수 : ', nullcnt)

결측야채개수 :  18


In [61]:
null_itemkind

[['건고추', '화건(1kg)', 8],
 ['건고추', '햇산화건(1kg)', 6070],
 ['양파', '양파(1kg)', 76],
 ['양파', '햇양파(1kg)', 6422],
 ['쌀', '일반계(1kg)', 29],
 ['쌀', '햇일반계(1kg)', 6902],
 ['감자', '대지마(1kg)', 3086],
 ['배추', '봄(1kg)', 5027],
 ['배추', '고랭지(1kg)', 4748],
 ['배추', '월동(1kg)', 2856],
 ['배추', '가을(1kg)', 4327],
 ['무', '봄(1kg)', 5107],
 ['무', '고랭지(1kg)', 4869],
 ['무', '월동(1kg)', 1299],
 ['무', '가을(1kg)', 3690],
 ['깐마늘(국산)', '햇깐마늘(대서)(1kg)', 169],
 ['깐마늘(국산)', '깐마늘(남도)(1kg)', 4],
 ['깐마늘(국산)', '햇깐마늘(남도)(1kg)', 169]]

목록 확인 : 특정 시기에만 생산되는 야채(햇양파 등)

In [62]:
# 특정 10월에만 가격이 없음
condi_nm = (data['itemnm']=='쌀') & (data['kindnm']=='일반계(1kg)')
condi_null = data['price']=='-'
data[condi_nm & condi_null]

,itemnm,kindnm,regdate,price
213664,쌀,일반계(1kg),1996-10-29,-
213665,쌀,일반계(1kg),1996-10-30,-
213666,쌀,일반계(1kg),1996-10-31,-
214953,쌀,일반계(1kg),1996-10-29,-
214954,쌀,일반계(1kg),1996-10-30,-
214955,쌀,일반계(1kg),1996-10-31,-
219600,쌀,일반계(1kg),2004-10-11,-
219601,쌀,일반계(1kg),2004-10-12,-
219602,쌀,일반계(1kg),2004-10-13,-
219603,쌀,일반계(1kg),2004-10-14,-
